Launching a project to create a crosstab app based on python.
<br>
It will have
- a 'data processing' module where user can transform the data
- a modeling module where use can select a particular model for the data

Rafael Parra Hernandez Nov 2024

# No constant variance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


# Libraries for modeling using sklearn
from sklearn.linear_model import LinearRegression

# Libraries for modeling using statsmodels (offers a little bit more of functionality)
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_reset

# Create heteroscedastic data (variance of y increases with X)
X = np.linspace(1, 10, 100)
y = (X ** 1).flatten() + np.random.normal(0, X, 100)  # Increasing variance as X increases
# Transform the data a tiny bit
X = X.flatten()

############################
# Make a dataframe
data = pd.DataFrame({'X': X, 'y': y})

# Fit linear regression model using the DataFrame
lr = LinearRegression()
lr.fit(data[['X']], data['y'])

# Make predictions and calculate residuals
data['y_pred'] = lr.predict(data[['X']])
data['residuals'] = data['y'] - data['y_pred']


fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))

# Plot residuals to check homoscedasticity
ax1.scatter(data['X'], data['residuals'])
ax1.axhline(0, color='red', linestyle='--')
ax1.set_xlabel("X")
ax1.set_ylabel("Residuals")
ax1.set_title("Residuals Plot (Violation of Homoscedasticity)")


# Now that we know variance is not constant (see above figure: residuals fan out)
# How to fix it?
# Apply log transformation to stabilize variance
y_transformed = np.log(data['y'] - data['y'].min() + 1)  # Log transformation to avoid negative values

# Fit linear regression to transformed data
lr.fit(data[['X']], y_transformed)
y_pred_transformed = lr.predict(data[['X']])

# Plot residuals after transformation
residuals_transformed = y_transformed - y_pred_transformed
ax2.scatter(data['X'], residuals_transformed)
ax2.axhline(0, color='red', linestyle='--')
ax2.set_xlabel("X")
ax2.set_ylabel("Residuals")
ax2.set_title("Residuals Plot After Transformation")

# Show the combined figure
plt.show()


# NON-LINEARITY

In [ ]:
# Code to test for linearity under the assumption that a linear model has been selected
# for modeling

# Some libraries for data manipulation & visualization
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Libraries for modeling using sklearn
from sklearn.linear_model import LinearRegression

# Libraries for modeling using statsmodels (offers a little bit more of functionality)
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_reset

# Create data that violates linearity
np.random.seed(0)
X = np.random.rand(100, 1) * 10  # Independent variable
y = (X ** 2).flatten() + np.random.normal(0, 5, X.shape[0])  # Dependent variable with a quadratic relationship
# Transform the data a tiny bit
X = X.flatten()

############################
# Make a dataframe
data = pd.DataFrame({'X': X, 'y': y})

lr = LinearRegression()
lr.fit(data[['X']], data['y'])
y_pred = lr.predict(data[['X']])

################################################
# Is it a good model? do the residuals plot are around zero?
# Plot residuals
residuals = data['y'] - y_pred
plt.scatter(data['X'], residuals)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("X")
plt.ylabel("Residuals")
plt.title("Residuals Plot (Violation of Linearity) -- SkLearn")
plt.show()


# Fit a linear model using API from statsmodels
linear_model = ols('y ~ X', data=data).fit()

# Ramsey's RESET Test
linear_model.fittedvalues = linear_model.fittedvalues.to_numpy()
reset_test_result = linear_reset(linear_model, power=2, use_f=True)


# Print Ramsey's RESET Test result
print("Ramsey's RESET Test p-value:", reset_test_result.pvalue)
if reset_test_result.pvalue < 0.05:
    print("Evidence of non-linearity (reject linearity).")
else:
    print("No evidence against linearity (fail to reject linearity).")

# Now that we know variance is not constant (see above figure: residuals fan out)
# How to fix it?
# Apply log transformation to stabilize variance
y_transformed = np.log(data['y'] - data['y'].min() + 1)  # Log transformation to avoid negative values

# Fit linear regression to transformed data
lr.fit(data[['X']], y_transformed)
y_pred_transformed = lr.predict(data[['X']])
# Predict as original units
y_original_units = np.exp(y_transformed) + data['y'].min() - 1


# Prepare data for statsmodels
data = pd.DataFrame({'X': X, 'y': y_transformed})
# Fit a linear model
linear_model = ols('y ~ X', data=data).fit()

# Ramsey's RESET Test
linear_model.fittedvalues = linear_model.fittedvalues.to_numpy()
reset_test_result = linear_reset(linear_model, power=2, use_f=True)
# Print Ramsey's RESET Test result
print("\nAFTER TRANSFORMATION-- Ramsey's RESET Test p-value:", reset_test_result.pvalue)
if reset_test_result.pvalue < 0.05:
    print("Evidence of non-linearity (reject linearity).")
else:
    print("No evidence against linearity (fail to reject linearity).")

# Print the equation of the model from scikit-learn's LinearRegression
print("\nScikit-Learn LinearRegression Model Equation:")
print(f"y = {lr.intercept_} + " + " + ".join([f"{coef} * X{i+1}" for i, coef in enumerate(lr.coef_)]))

# Computing confidence interval (switching to a statsmodels)
X_with_const = sm.add_constant(data['X'])  # Add constant for intercept
ols_model = sm.OLS(y_transformed, X_with_const).fit()

# Get the confidence intervals for the predictions
confidence_level = 0.95
predictions = ols_model.get_prediction(X_with_const)
conf_int = predictions.conf_int(alpha=1-confidence_level)

print(f"\n{int(confidence_level * 100)}% Confidence Interval for predictions:")
# Print the equation of the model from statsmodels' OLS
print("\nStatsmodels OLS Model Equation:")
print(f"y = {ols_model.params.iloc[0]} + " + " + ".join([f"{coef} * X{i}" for i, coef in enumerate(ols_model.params.iloc[1:], start=1)]))
